
## Step 1: Setup and Imports


In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv
import cv2
import requests
from PIL import Image
from io import BytesIO

from pokedex.utils_dataset import create_dataset


## Image Download + Cropping


In [ ]:
s_id = 'dv1'
p_id = '1'

# Get metadata and image URL from TCG API
url = f'https://api.pokemontcg.io/v2/cards/{s_id}-{p_id}'
response = requests.get(url).json()
image_url = response['data']['images']['large']

# Download and open the image
response_card = requests.get(image_url)
if response_card.status_code == 200:
    image = Image.open(BytesIO(response_card.content))
else:
    print(f"Failed to retrieve image. HTTP Status code: {response_card.status_code}")

# Convert to grayscale
grayImage = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)
plt.imshow(grayImage, cmap='gray')

# Crop corners (percent-based approach)
width, height = image.size
bottomleft = image.crop((width * 0.75, height * 0.93, width, height))
bottomright = image.crop((0, height * 0.93, width * 0.25, height))


## Dataset Assembly (Set Looping + Labeling)

In [ ]:
setinfo = np.array([
    ['dv1', '21', 'Dragon Vault', 'right'],
    ['swsh9', '186', 'Brilliant Stars', 'left'],
    ['swsh45', '73', 'Shining Fates', 'left'],
    ['swsh6', '233', 'Chilling Reign', 'left'],
    ['swsh12pt5', '160', 'Crown Zenith', 'left'],
    ['xy1', '146', 'XY', 'right'],
    ['xy2', '110', 'Flashfire', 'right'],
    ['xy3', '114', 'Furious Fists', 'right'],
    ['g1', '117', 'Generations', 'right'],
    ['xy4', '124', 'Phantom Forces', 'right'],
    ['xy6', '112', 'Roaring Skies', 'right'],
    ['xy7', '100', 'Ancient Origins', 'right'],
    ['dp1', '130', 'Diamond & Pearl', 'right'],
    ['dp2', '124', 'Mysterious Treasures', 'right'],
    ['sm4', '126', 'Crimson Invasion', 'left'],
    ['swsh10', '216', 'Astral Radiance', 'left'],
    ['sv4', '266', 'Paradox Rift', 'left'],
    ['sv3pt5', '207', '151', 'left'],
    ['sv3', '230', 'Obsidian Flames', 'left'],
    ['sv2', '279', 'Paldea Evolved', 'left']
])

dataset_df = pd.DataFrame(columns=['corner', 'position', 'set_id', 'set_name'], index=[0])
k = 0

for j in range(setinfo.shape[0]):
    s_id = setinfo[j, 0]
    print(f'On-going set: {s_id}')

    for i in range(1, int(setinfo[j, 1]) + 1):
        url = f'https://api.pokemontcg.io/v2/cards/{s_id}-{str(i)}'
        response = requests.get(url).json()
        image_url = response['data']['images']['large']
        response_card = requests.get(image_url)

        if response_card.status_code == 200:
            image = Image.open(BytesIO(response_card.content))
            width, height = image.size

            bottomleft = image.crop((width * 0.75, height * 0.93, width, height))
            bottomright = image.crop((0, height * 0.93, width * 0.25, height))

            graybottomleft = cv2.cvtColor(np.array(bottomleft), cv2.COLOR_BGR2GRAY)
            graybottomright = cv2.cvtColor(np.array(bottomright), cv2.COLOR_BGR2GRAY)

            if setinfo[j, 3] == 'left':
                dataset_df.loc[k] = [graybottomleft, 'left', setinfo[j, 0], setinfo[j, 2]]
                k += 1
                dataset_df.loc[k] = [graybottomright, 'right', 'no', 'no']
                k += 1
            elif setinfo[j, 3] == 'right':
                dataset_df.loc[k] = [graybottomleft, 'left', 'no', 'no']
                k += 1
                dataset_df.loc[k] = [graybottomright, 'right', setinfo[j, 0], setinfo[j, 2]]
                k += 1
        else:
            print(f"Failed to retrieve image. HTTP Status code: {response_card.status_code}")